In [ ]:
import numpy as np
import pandas as pd
import pygmo as pg
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 500)

os.environ['KERAS_BACKEND'] = 'tensorflow'
from keras.models import load_model
from sklearn.externals import joblib 
from config import MAX_DISCOUNT, FEATURES

%matplotlib inline
%load_ext line_profiler

In [ ]:
df=pd.read_csv('training_data_randn.csv', nrows=50000)
cols_x = FEATURES
col_y = 'is_deal'
col_p = 'Probability'
print(df.shape)

In [ ]:
class propensity_predictor:
    def __init__(self, f_scalar_pickle, f_model_h5):
        self.f_scalar_pickle = f_scalar_pickle
        self.f_model_h5 = f_model_h5
        
    def predict(self, X):
        if not hasattr(self, 'model'):
            self.scalar = joblib.load(self.f_scalar_pickle)
            self.model = load_model(self.f_model_h5)            
        X_s = self.scalar.transform(X)
        y = self.model.predict(X_s)[:, 0]
        return y
    
    def __copy__(self):
        newone = type(self)(self.f_scalar_pickle, self.f_model_h5)
        return newone
    
    def __deepcopy__(self, memo):
        newone = type(self)(self.f_scalar_pickle, self.f_model_h5)
        return newone
    
m = propensity_predictor('x_scalar_randn.pkl', 'propensity_model_randn.h5')

In [ ]:
%%timeit
m.predict(df[cols_x])

In [ ]:
df.loc[:, 'pred'] = m.predict(df[cols_x])

# Background

Linear programming vs non-linear programming

Optimization algorithms: local vs heuristic

# User Defined Problem (UDP)

## Objectives

In [ ]:
v_space = 0.01
f_discounts = np.arange(0, MAX_DISCOUNT + v_space, v_space)

cols_grpby = ['product_id']

scan_propensity = []
scan_revenue = []
scan_profit = []

for f in f_discounts:
    X = df[cols_x].values
    X[:, -1] = df['RRP']*f 
    col_discount = 'Discount_{:.02f}'.format(f)
    col_pred = 'y_pred_{:.02f}'.format(f)
    df.loc[:, col_discount] = X[:, -1]
    df.loc[:, col_pred] = m.predict(X)
    
    # Prospensity
    s_prop = df.groupby(cols_grpby).apply(lambda df: np.mean(df[col_pred])).rename(f)
    s_reve = df.groupby(cols_grpby).apply(lambda df: np.dot(df['RRP'] - df[col_discount], df[col_pred])).rename(f)
    s_prof = df.groupby(cols_grpby).apply(lambda df: np.dot(df['RRP'] - df[col_discount] - df['Cost'], df[col_pred])).rename(f)

    scan_propensity.append(s_prop)
    scan_revenue.append(s_reve)
    scan_profit.append(s_prof)

In [ ]:
f, ax = plt.subplots(3, 1, figsize=(16,18))
pd.concat(scan_propensity, axis=1).T.plot(ax = ax[0],  title='Propensity', grid=True)
pd.concat(scan_revenue, axis=1).T.plot(ax = ax[1],  title='Revenue', grid=True)
pd.concat(scan_profit, axis=1).T.plot(ax = ax[2],  title='Profit', grid=True)

## Single-objective optimization

Find out the discount amount for each product in order to maximize revenue

In [ ]:
class optimal_discount_per_product_for_max_profit:
    def __init__(self, df, model): 
        self.cols_x = FEATURES
        self.df = df     
        self.p_ids = df['product_id'].unique()
        self.p_ids.sort()
        self.n_products = len(self.p_ids)
        self.model = model
        
    def fitness(self, dv): # fitness given decision vector (dv)
        prod_discount = dict(zip(self.p_ids, dv))
        v_dis = self.df['product_id'].map(prod_discount) * self.df['RRP']
        X = self.df[self.cols_x].values
        X[:, -1] = v_dis
        y_prob = self.model.predict(X)
        profit = np.dot(self.df['RRP'] - v_dis - self.df['Cost'], y_prob)  
        return [-profit/1e6,]   

    def get_bounds(self): # box bounds of decision vector (dv)
        return ([0,]*self.n_products, [MAX_DISCOUNT, ]*self.n_products)

Notes:
* PyGMO2 assume minimization in every objective
* Default assume single-objective, all continuous decision vectors
* Return of fitness() must be a list, even for single-objective problems

In [ ]:
pp_profit = optimal_discount_per_product_for_max_profit(df, m)

In [ ]:
prob = pg.problem(pp_profit)
algo = pg.algorithm(pg.de(gen = 50))
pop = pg.population(prob,10)
algo.set_verbosity(2)

In [ ]:
pop = algo.evolve(pop)

In [ ]:
best_x_rev = pop.champion_x
best_f_rev = pop.champion_f
print(best_x_rev, best_f_rev) 

In [ ]:
log = algo.extract(pg.de).get_log()

In [ ]:
log

## Multi-objective optimization

In [ ]:
class optimal_discount_per_product_for_max_revenue_min_cost:
    def __init__(self, df, model): 
        self.cols_x = FEATURES
        self.df = df
        self.p_ids = df['product_id'].unique()
        self.p_ids.sort()
        self.n_products = len(self.p_ids)
        self.model = model        
   
    def get_nobj(self):
        return 2
    
    def fitness(self, dv): # fitness given decision vector (dv)
        prod_discount = dict(zip(self.p_ids, dv))
        v_dis = self.df['product_id'].map(prod_discount) * self.df['RRP']
        X = self.df[self.cols_x].values
        X[:, -1] = v_dis
        y_prob = self.model.predict(X)
        revenue = np.dot(self.df['RRP'] - v_dis, y_prob)  
        cost = np.dot(self.df['Cost'], y_prob)  
        return [-revenue/1e6,cost/1e6]   # Minimization objectives
    
    def get_bounds(self): # box bounds of decision vector (dv)       
        return ([0,]*self.n_products, [MAX_DISCOUNT, ]*self.n_products)

In [ ]:
pp_rev_cost = optimal_discount_per_product_for_max_revenue_min_cost(df, m)

In [ ]:
prob = pg.problem(pp_rev_cost)
pop = pg.population(prob,64)

In [ ]:
algo = pg.algorithm(pg.nsga2(gen = 1))
# algo.set_verbosity(1)

In [ ]:
refpoint = [ -30, 30]
log_pop = []
hv = pg.hypervolume(pop).compute(refpoint)
log_pop.append((pop.get_x(), pop.get_f(), hv))
print(hv)
for i in range(129):
    pop = algo.evolve(pop)
    hv = pg.hypervolume(pop).compute(refpoint)
    log_pop.append((pop.get_x(), pop.get_f(), hv))
    if np.log2(i+1).is_integer():
        print(i, hv)


In [ ]:
fits, vectors = -pop.get_f(), pop.get_x()
ndf, dl, dc, ndr = pg.fast_non_dominated_sorting(fits)

In [ ]:
idx_sorted = fits.argsort(0)[:, 0]

In [ ]:
for id in idx_sorted:
    print(fits[id], vectors[id].round(3))

In [ ]:
help(df.plot)

In [ ]:
def plot_pareto_frontiers(fs, **kwarg):
    df = pd.DataFrame(fs, columns = ('Revenue', 'Cost')).sort_values('Revenue')   
    df.loc[:, 'pareto_group'] = pg.fast_non_dominated_sorting(df.values)[3]
    df['Revenue'] = -df['Revenue']
    df.groupby('pareto_group').plot('Revenue', 'Cost', style='o-', legend=False, **kwarg, ax=plt.gca())
    plt.gca().set_ylabel('Cost')
    return
   
x_max = np.array([pop[1] for pop in log_pop])[:,:, 0].max()+1
x_min = np.array([pop[1] for pop in log_pop])[:,:, 0].min()-1
y_max = np.array([pop[1] for pop in log_pop])[:,:, 1].max()+1
y_min = np.array([pop[1] for pop in log_pop])[:,:, 1].min()-1
axis_range = ((x_min, x_max), (y_min, y_max))
for i, (dvs, fs, hv) in enumerate(log_pop):
    if np.log2(i+1).is_integer():
        plt.cla()
        ax = plot_pareto_frontiers(fs, xlim=(-x_max, -x_min), ylim=(y_min, y_max), figsize=(8,6), title='Evolution %d'%(i+1)) 
        plt.savefig('test_%d.png'%(np.log2(i+1)))


## Hypervolume

* Reference point
* For dimensionality>1 only


## Decorator meta-problem
https://esa.github.io/pagmo2/docs/python/tutorials/udp_meta_decorator.html


In [ ]:
??m.model._make_predict_function

## Parallelization

Island

Run evolutions on 
* multi-threading
* multi-processing
* cluster

Archipelago


Keras: 
model._make_predict_function 
https://stackoverflow.com/questions/40850089/is-keras-thread-safe

In [ ]:
algo = pg.algorithm(pg.nsga2(gen = 1))
prob = pg.problem(pp_rev_cost)
pop = pg.population(prob,16)

In [ ]:
isl_0 = pg.island(algo = pg.de(10), prob = pg.ackley(5), size=20, udi=pg.thread_island())

## Optimization with constraint

## Integer problem

## Optimization with gradient/hessian

## Logging during training

log = algo.extract(pg.nlopt).get_log()
from matplotlib import pyplot as plt 
plt.semilogy([line[0] for line in log], [line[1] for line in log], label = "obj") 
plt.semilogy([line[0] for line in log], [line[3] for line in log], label = "con")
